# Introduction #

Load the *Housing* dataset.

# Dropout #

### 1) Define model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[11]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])

Add 50% dropout layers after each of the hidden layers.

In [ ]:
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[11]),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1),
])

Now run this to train the model.

In [ ]:
model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae', 'mse'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=20,
)

# Batchnorm #

Add batch normalization after each hidden layer.

In [ ]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[11]),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1),
])

Now run this to train the model.

In [ ]:
model.compile(
    optimizer='adam',
    loss='mae',
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=20,
)

Did it require fewer epochs?

# Conclusion #